# Tensorflow를 이용한 MNIST 실습 

## 핵심은 정확도 비교해보기!
#### 1) hidden layer 개수를 조절하면서
#### 2) layer 의 node 개수를 조절하면서
#### 3) epoch의 수를 조절하면서

In [26]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [27]:
# tensor graph reset 함수
tf.reset_default_graph()

In [28]:
mnist = input_data.read_data_sets('data/mnist/', one_hot=True)   #이미지 데이터 저장, one_hot incoding하기

Extracting data/mnist/train-images-idx3-ubyte.gz
Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


In [29]:
mnist.train, mnist.test

(<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet at 0x279577ed160>,
 <tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet at 0x27957957b00>)

In [30]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

neural networks 구축  
1) layer 수 조절해보기  
2) layer 마다의 node 수 조절해보기

In [34]:
W_1 = tf.Variable(tf.random_normal([784, 256]))   # Weight = 784*256
b_1 = tf.Variable(tf.random_normal([256]))        # bias = 256*1
hidden_1 = tf.sigmoid(tf.matmul(X, W_1) + b_1)     #hidden layer = X * W_1 + b_1 = WX -> Sigmoid

W_2 = tf.Variable(tf.random_normal([256, 10]))
b_2 = tf.Variable(tf.random_normal([10]))
hypo = tf.sigmoid(tf.matmul(hidden_1, W_2) + b_2)

In [35]:
cost = tf.reduce_mean(  # cost 계산
    tf.nn.softmax_cross_entropy_with_logits(
        logits=hypo, labels=y))
# 우리가 이미 살펴봤던 loss
# logits = 예측값, lables = 실제값

In [36]:
learning_rate = 1
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

predict : hypo > 0.5 크다면 1로 저장.  
hypo > 0.5 -> 논리식이기 때문에, 결과값이 True or False  
tf.cast("value", "변환할 type")
: value를 해당 type으로 형변환 : True -> 1, False -> 0

accuracy  
tf.equal(predict, y) : predict와 y 과 같니? -> 같으면 True, 다르면 False  
tf.cast : 1, 0 (array로 저장)
tf.reduce_mean : array 평균 : 전체중에 1이 몇 %있니?  
ex) [1, 1, 1, 1, 0] : 0.8

In [39]:
predict = tf.cast(hypo > 0.5, tf.float32) # 1 or 0    // hypo가 0.5보다 크면 true=1, 작으면 false = 0으로 cast해줌
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), tf.float32))   #predict랑 y가 같으면 1/다르면0을 모든 data에 해서 그걸 평균냄

3) epoch 수를 조절해보세요.

In [43]:
batch_size = 100  # 몇 개로 나눠서 돌릴꺼야?
epoches = 10   # 몇 바퀴 돌릴꺼야?

In [44]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # total_batch 수(업데이트 수) = 전체 데이터 개수 // batch크기
    total_batch = mnist.train.num_examples//batch_size     # 550 = 55000(n_tr)/100
    print('total_batch : ', total_batch)

    for epoch in range(epoches):   # epoch개수=10 만큼 돌릴꺼야 -> 총 5500번 update
        avg_cost = 0    # avg 초기화

        for i in range(total_batch):  # total batch수 만큼 for문 돌려서 optimizer를 누르자
            #training dataset
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)   # next_batch : 다음 batch로 넘어감


            sess.run(optimizer, feed_dict={X: batch_xs, y: batch_ys})
            c = sess.run(cost, feed_dict={X: batch_xs, y: batch_ys})

            avg_cost += c / total_batch

        print('{:2} : {:}'.format(epoch+1, avg_cost))
        
    print('Learning finished!')

    print('accuracy :', sess.run(accuracy, feed_dict={X: mnist.test.images, 
                                                  y: mnist.test.labels}))


total_batch :  550
 1 : 1.9969321205399224
 2 : 1.8408165062557564
 3 : 1.7832937249270324
 4 : 1.760353579521181
 5 : 1.746986901760103
 6 : 1.737656547589736
 7 : 1.7307951400496768
 8 : 1.7253239820220263
 9 : 1.7206392470273117
10 : 1.7168226200884031
Learning finished!
accuracy : 0.94932
